<a href="https://colab.research.google.com/github/mortaloxygenknight/Taxsea/blob/main/1031tif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install rasterio
import os
import numpy as np
import rasterio
from rasterio.mask import mask
from google.colab import drive
import glob
from scipy import ndimage
import matplotlib.pyplot as plt

# 挂载Google Drive
drive.mount('/content/drive')

def preprocess_dem(input_path, output_path):
    """
    对ASTER GDEM数据进行预处理
    参数:
    input_path: 输入文件路径
    output_path: 输出文件路径
    """
    with rasterio.open(input_path) as src:
        # 读取数据和元数据
        dem = src.read(1).astype(np.float32)  # 确保数据类型是float32
        profile = src.profile.copy()

        # 获取nodata值
        nodata_value = src.nodata
        if nodata_value is None:
            nodata_value = -9999  # 设置默认nodata值

        # 创建掩码
        valid_mask = dem != nodata_value

        # 1. 异常值处理
        # 定义合理的高程范围（根据研究区域调整）
        min_elevation = -500  # 最小合理高程
        max_elevation = 9000  # 最大合理高程

        # 将超出范围的值标记为nodata
        dem[~valid_mask] = nodata_value
        dem[(dem < min_elevation) & valid_mask] = nodata_value
        dem[(dem > max_elevation) & valid_mask] = nodata_value

        # 更新有效数据掩码
        valid_mask = dem != nodata_value

        # 2. 去除异常尖峰和凹坑
        # 只对有效数据进行中值滤波
        dem_filtered = dem.copy()
        dem_filtered[valid_mask] = ndimage.median_filter(dem[valid_mask], size=3)

        # 3. 填充小型数据空洞
        # 使用插值填充小的NoData区域
        if np.any(~valid_mask):
            # 只对小面积的nodata区域进行填充
            struct = ndimage.generate_binary_structure(2, 2)
            labeled_array, num_features = ndimage.label(~valid_mask, structure=struct)

            for label in range(1, num_features + 1):
                # 获取当前空洞的掩码
                hole_mask = labeled_array == label
                # 如果空洞面积小于100个像素，则进行填充
                if np.sum(hole_mask) < 100:
                    # 使用周围有效值的平均值填充
                    boundary = ndimage.binary_dilation(hole_mask) & valid_mask
                    if np.any(boundary):
                        fill_value = np.mean(dem[boundary])
                        dem_filtered[hole_mask] = fill_value

        # 4. 平滑处理
        # 只对有效数据进行高斯滤波
        dem_smooth = dem_filtered.copy()
        valid_mask = dem_smooth != nodata_value
        dem_smooth[valid_mask] = ndimage.gaussian_filter(dem_filtered[valid_mask], sigma=1)

        # 保存处理后的数据
        profile.update(
            dtype=rasterio.float32,
            nodata=nodata_value
        )

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(dem_smooth.astype(rasterio.float32), 1)

        return dem_smooth

def batch_process_dems(input_folder, output_folder):
    """
    批量处理文件夹中的所有DEM文件
    """
    # 创建输出文件夹（如果不存在）
    os.makedirs(output_folder, exist_ok=True)

    # 获取所有tif文件
    dem_files = glob.glob(os.path.join(input_folder, '*.tif'))
    print(f"Found {len(dem_files)} DEM files")

    # 批量处理
    for dem_file in dem_files:
        print(f"Processing: {os.path.basename(dem_file)}")
        output_file = os.path.join(output_folder, f"processed_{os.path.basename(dem_file)}")
        try:
            processed_dem = preprocess_dem(dem_file, output_file)

            # 可视化处理前后的结果
            with rasterio.open(dem_file) as src:
                original_dem = src.read(1)

            plt.figure(figsize=(15, 5))

            plt.subplot(121)
            plt.imshow(original_dem, cmap='terrain')
            plt.title('Original DEM')
            plt.colorbar()

            plt.subplot(122)
            plt.imshow(processed_dem, cmap='terrain')
            plt.title('Processed DEM')
            plt.colorbar()

            plt.savefig(os.path.join(output_folder, f"comparison_{os.path.basename(dem_file)}.png"))
            plt.close()

            print(f"Successfully processed: {os.path.basename(dem_file)}")

        except Exception as e:
            print(f"Error processing {dem_file}: {str(e)}")
            print(f"Details: {type(e).__name__}")
            continue

# 设置输入和输出文件夹路径
input_folder = '/content/drive/My Drive/Colab Notebooks/1031/dem'  # 调整为您的实际路径
output_folder = '/content/drive/My Drive/Colab Notebooks/1031/dem_processed'  # 处理后的文件存放路径

# 执行批处理
batch_process_dems(input_folder, output_folder)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 32 DEM files
Processing: ASTWBDV001_N37E136_dem.tif
Successfully processed: ASTWBDV001_N37E136_dem.tif
Processing: ASTWBDV001_N37E137_dem.tif
Successfully processed: ASTWBDV001_N37E137_dem.tif
Processing: ASTWBDV001_N37E138_dem.tif
Successfully processed: ASTWBDV001_N37E138_dem.tif
Processing: ASTWBDV001_N37E139_dem.tif
Successfully processed: ASTWBDV001_N37E139_dem.tif
Processing: ASTWBDV001_N36E136_dem.tif
Successfully processed: ASTWBDV001_N36E136_dem.tif
Processing: ASTWBDV001_N36E137_dem.tif
Successfully processed: ASTWBDV001_N36E137_dem.tif
Processing: ASTWBDV001_N36E138_dem.tif
Successfully processed: ASTWBDV001_N36E138_dem.tif
Processing: ASTWBDV001_N36E139_dem.tif
Successfully processed: ASTWBDV001_N36E139_dem.tif
Processing: ASTWBDV001_N35E136_dem.tif
Successfully processed: ASTWBDV001_N35E136_dem.tif
Processing: ASTWBDV001_N35E137_dem.tif
Suc